In [ ]:
from core.utils import Tibanna
from core import ff_utils
from uuid import UUID
import ast

env = 'fourfront-webprod'
tibanna = Tibanna(env=env)
ff = ff_utils.fdn_connection(key=tibanna.ff_keys)

wfrB= ['5523499f-3c53-4ee7-a635-1de2417216d4/']


for uuid in wfrB:
    item = ff_utils.get_metadata(uuid, connection=ff)
    print item['display_title']
    assert item['display_title'].startswith('hi-c-processing-partb/')
    inputs = [i['value'] for i in item['input_files'] if i['workflow_argument_name'].startswith('input_pairs')]
    print(inputs)
    for ix in inputs:
        inp_pair = ff_utils.get_metadata(ix, connection=ff)
        parta = inp_pair['input_wffr']
        
        my_id = ix['uuid']
        response = [ixx['uuid'] for ixx in requests.get(url(my_id), auth=auth, headers=headers).json()['experiments']]
        exp_list.extend(response)
    exp_list = list(set(exp_list))
    outputs = [i['value'] for i in item['output_files']]




In [ ]:
import json
import copy
def record_object(uuid, store, item_uuids):
    #keep list of fields that only exist in frame embedded (revlinks) that you want connected
    
#     add_from_embedded = {'file_fastq':['workflow_run_inputs', 'workflow_run_outputs'],
#                          'file_processed':['workflow_run_inputs', 'workflow_run_outputs']
#                         }
    add_from_embedded = {}
   
    #find schema name, store as obj_key, create empty list if missing in store

    obj_type = ff_utils.get_metadata(uuid, connection=ff)['@type'][0]

    try:
        obj_key = schema_name[obj_type]
    except:
        #print 'CAN NOT FIND', obj_type, uuid
        return store, item_uuids
    if obj_key not in store:
        store[obj_key] = []

    add_uuids = []
    resp = ff_utils.get_metadata(uuid, connection=ff, frame = 'raw')
    
    if resp['status'] not in ['current', 'released', 'in review by lab', 'uploaded']:
        print obj_key, uuid, resp['status']
        
    # add raw frame to store and uuid to list
    if uuid not in item_uuids:
        store[obj_key].append(resp)
        item_uuids.append(uuid)
    # this case should not happen actually
    else:
        return store, item_uuids
    
    fields_to_check = copy.deepcopy(resp)
  
    # check if any field from the embedded frame is required
    add_fields = add_from_embedded.get(obj_key)
    if add_fields:
        emb_resp = ff_utils.get_metadata(uuid, connection=ff, frame = 'embedded')
        for a_field in add_fields:
            field_val = emb_resp.get(a_field)
            if field_val:
                fields_to_check[a_field] = field_val
        
    for key, value in fields_to_check.iteritems():
        # uuid formatted fields to skip
        if key == 'uuid':
            continue
        uuid_in_val = find_uuids(value)
        
        for a_uuid in uuid_in_val:
            if a_uuid not in item_uuids:
                store, item_uuids = record_object(a_uuid, store, item_uuids)   
    return store, item_uuids
            
            
store_dict = {} 
item_uuids = []
store, uuids = record_object('da387ade-8822-4009-9269-90e2302c09d2', store_dict, item_uuids)
print(len(uuids))
print(len(list(set(uuids))))

In [ ]:
## This part is only for releasing
counter = 0
for a_type in store:
    for raw_data in store[a_type]:
        if raw_data['status'] in ['deleted']: 
            print 'deleted data', a_type ,raw_data['uuid']
            continue        
        if raw_data['status'] not in ['current', 'released']:    
            counter +=1
            print a_type, raw_data['status'], raw_data['uuid']
            patch_data = {'status': "released"}
            ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,connection=ff)
print counter